# 4.Training set generation [Task1 , Task2]

In [1]:
import os
import pathlib
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from haversine import haversine
import seaborn as sns
import json
from shapely.geometry import Point
sns.set()

In [2]:
#Font
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
matplotlib.rcParams['axes.unicode_minus']=False

font_fname = '/opt/app-root/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()
plt.rcParams["font.family"] = font_family

In [3]:
path = pathlib.Path('./data')

In [4]:
주정차단속 = pd.read_csv(os.path.join(path,'1.오산시_주정차단속(2018~2020).csv'),encoding="utf-8")
어린이교통사고_격자 = gpd.read_file(os.path.join(path,'2.오산시_어린이교통사고_격자.geojson'))
차량등록현황_격자 = gpd.read_file(os.path.join(path,'3.오산시_차량등록현황_격자.geojson'))
거주인구격자_총인구 = gpd.read_file(os.path.join(path,'4.오산시_연령별_거주인구격자(총인구).geojson'))
거주인구격자_유소년 = gpd.read_file(os.path.join(path,'5.오산시_연령별_거주인구격자(유소년).geojson'))
거주인구격자_생산가능 = gpd.read_file(os.path.join(path,'6.오산시_연령별_거주인구격자(생산가능인구).geojson'))
거주인구격자_고령 = gpd.read_file(os.path.join(path,'7.오산시_연령별_거주인구격자(고령).geojson'))
유동인구 = pd.read_csv(os.path.join(path,'8.오산시_유동인구(2019).csv'),encoding="utf-8")
어린이보호구역 = pd.read_csv(os.path.join(path,'9.오산시_어린이보호구역.csv'),encoding="utf-8")
학교위치정보 = pd.read_csv(os.path.join(path,'10.오산시_학교위치정보.csv'),encoding="utf-8")
초등학교_통학구 = gpd.read_file(os.path.join(path,'11.오산시_초등학교_통학구.geojson'))
중학교_학군 = gpd.read_file(os.path.join(path,'12.오산시_중학교_학군.geojson'))
어린이집_유치원현황 = pd.read_csv(os.path.join(path,'13.오산시_어린이집_유치원현황.csv'),encoding="utf-8")
기상데이터 = pd.read_csv(os.path.join(path,'14.오산시_기상데이터(2010~2019).csv'),encoding="utf-8")
무인교통단속카메라 = pd.read_csv(os.path.join(path,'15.오산시_무인교통단속카메라.csv'),encoding="utf-8")
도로안전표지 = pd.read_csv(os.path.join(path,'16.오산시_도로안전표지표준데이터.csv'),encoding="utf-8")
횡단보도 = gpd.read_file(os.path.join(path,'17.오산시_횡단보도.geojson'))
과속방지턱 = pd.read_csv(os.path.join(path,'18.오산시_과속방지턱표준데이터.csv'),encoding="utf-8")
신호등 = gpd.read_file(os.path.join(path,'19.오산시_신호등.geojson'))
CCTV설치현황 = pd.read_csv(os.path.join(path,'20.오산시_CCTV설치현황.csv'),encoding="utf-8")
인도 = gpd.read_file(os.path.join(path,'21.오산시_인도.geojson'))
버스정류장 = pd.read_csv(os.path.join(path,'22.오산시_버스정류장.csv'),encoding="utf-8")
상세도로망 = gpd.read_file(os.path.join(path,'23.오산시_상세도로망_LV6.geojson'))
시간대별_추정교통량 = pd.read_csv(os.path.join(path,'24.평일_전일,시간대별_오산시_추정교통량_Level6.csv'),encoding="utf-8")
혼잡빈도강도 = pd.read_csv(os.path.join(path,'25.평일_전일_오산시_혼잡빈도강도_Level6.csv'),encoding="utf-8")
혼잡시간강도 = pd.read_csv(os.path.join(path,'26.평일_전일_오산시_혼잡시간강도_Level6.csv'),encoding="utf-8")
도로명주소_건물 = gpd.read_file(os.path.join(path,'27.오산시_도로명주소_건물.geojson'))
건물연면적_격자 = gpd.read_file(os.path.join(path,'28.오산시_건물연면적_격자.geojson'))
체육시설현황 = pd.read_csv(os.path.join(path,'29.오산시_체육시설현황.csv'),encoding="utf-8")
학원_및_교습소_현황 = pd.read_csv(os.path.join(path,'30.오산시_학원_및_교습소_현황.csv'),encoding="utf-8")
법정경계_시군구 = gpd.read_file(os.path.join(path,'31.오산시_법정경계(시군구).geojson'))
행정경계_읍면동 = gpd.read_file(os.path.join(path,'32.오산시_행정경계(읍면동).geojson'))
법정경계_읍면동 = gpd.read_file(os.path.join(path,'33.오산시_법정경계(읍면동).geojson'))
지적도 = gpd.read_file(os.path.join(path,'34.오산시_지적도.geojson'))

관심학원_현황= pd.read_csv(os.path.join(path,'관심학원_현황.csv'),encoding="utf-8")
초등학교위치정보= pd.read_csv(os.path.join(path,'초등학교위치정보.csv'),encoding="utf-8")
평균유동인구= pd.read_csv(os.path.join(path,'평균유동인구.csv'),encoding="utf-8")

# [과제 1] 모델링을 위한 feature column 생성 : 원핫인코딩 포함 총 85개 칼럼

__어린이 교통사고 격자 데이터를 기본으로 칼럼을 누적__

In [13]:
data = 어린이교통사고_격자[어린이교통사고_격자.accident_cnt>0].reset_index().copy()
data = data.drop(["index"],axis=1)

In [14]:
data.head(3)

,gid,accident_cnt,geometry
0,다사561083,1,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ..."
1,다사564096,1,"MULTIPOLYGON (((127.00880 37.18416, 127.00879 ..."
2,다사566102,1,"MULTIPOLYGON (((127.01101 37.18958, 127.01101 ..."


> __사고 격자 내 평균 유동인구__  
(col_name = 유동인구_sum)

In [ ]:
def ppl_flow(geo_obj):
    
    global 평균유동인구
    
    ret_cnt = 0
    
    for (lon,lat,cnt) in 평균유동인구.iloc:
        current_point = Point(lon,lat)
        if geo_obj.contains(current_point):
            ret_cnt+=cnt
    return ret_cnt

data['유동인구_sum'] = data['geometry'].apply(ppl_flow)

> __사고 격자 주변 유소년 인구 (100m , 200m)__  
(col_name = 유소년_count_100 , 유소년_count_200)

In [ ]:
def young_living_nearby(geo_obj,rad):
    
    global 거주인구격자_유소년
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_,cnt,square) in 거주인구격자_유소년[거주인구격자_유소년["val"]>0].iloc:
        c_y,c_x = square.centroid.coords[0]
        
        if haversine((x,y),(c_x,c_y),unit="m") <= rad:
            ret_cnt+=cnt
            
    return ret_cnt

data['유소년인구_count_100'] = data['geometry'].apply(lambda x:young_living_nearby(x,100))
data['유소년인구_count_200'] = data['geometry'].apply(lambda x:young_living_nearby(x,200))

> __사고 격자 주변 생산가능,고령,총 인구 (100m)__  
(col_name = 생산가능인구_count_100 , 고령인구_count_100 , 총인구_count_100)

In [ ]:
def other_living_nearby(geo_obj,other):
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_,cnt,square) in other[other["val"]>0].iloc:
        c_y,c_x = square.centroid.coords[0]
        
        if haversine((x,y),(c_x,c_y),unit="m") <= 100:
            ret_cnt+=cnt
            
    return ret_cnt

data['생산가능인구_count_100'] = data['geometry'].apply(lambda x:other_living_nearby(x,거주인구격자_생산가능))
data['고령인구_count_100'] = data['geometry'].apply(lambda x:other_living_nearby(x,거주인구격자_고령))
data['총인구_count_100'] = data['geometry'].apply(lambda x:other_living_nearby(x,거주인구격자_총인구))

> __사고 격자 주변 학원 수 (100m , 300m)__  
(col_name = 학원수_count_100 , 학원수_count_300)

In [ ]:
def hakwon_nearby(geo_obj,rad):
    
    global 관심학원_현황
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_1,_2,c_y,c_x) in 관심학원_현황.iloc:
        if haversine((x,y),(c_x,c_y),unit="m") <= rad:
            ret_cnt+=1
    
    return ret_cnt

data['학원수_count_100'] = data['geometry'].apply(lambda x : hakwon_nearby(x,100))
data['학원수_count_300'] = data['geometry'].apply(lambda x : hakwon_nearby(x,300))

> __사고 격자 소속 동__  
(col_name = 동_구분)

In [ ]:
def dong_assigned(geo_obj):
    
    global 법정경계_읍면동
    
    point = geo_obj.centroid
    
    for line in 법정경계_읍면동.iloc:
        dong = line.geometry
        if dong.contains(point):
            return line.EMD_KOR_NM
    return np.nan

data["동_구분"] = data["geometry"].map(dong_assigned)
data = data.join(pd.get_dummies(data["동_구분"],prefix="동_")) # One-Hot-Encoding 칼럼 별도 추가

> __사고 격자 소속 초등학교 통학구__  
(col_name = 통학구_구분)

In [ ]:
def tonghakgu(geo_obj):
    
    global 초등학교_통학구
    
    point = geo_obj.centroid
    
    for line in 초등학교_통학구.iloc:
        dong = line.geometry
        if dong.contains(point):
            return line["학구명"]
    return np.nan

data["통학구_구분"] = data["geometry"].map(tonghakgu)
data = data.join(pd.get_dummies(data["통학구_구분"],prefix="통학구_")) # One-Hot-Encoding 칼럼 별도 추가

> __사고 격자 내 신호등 수__  
(col_name = 신호등_sum)

In [ ]:
def traffic_light(geo_obj):
    
    global 신호등

    cnt = 0
    for line in 신호등.iloc:
        _,obj = line
        if geo_obj.contains(obj):
            cnt+=1
    return cnt

data["신호등_sum"] = data["geometry"].map(traffic_light)

> __사고 격자 내 횡단보도 수__  
(col_name = 횡단보도_sum)

In [ ]:
def cross_walk(geo_obj):
    
    global 횡단보도

    cnt = 0
    for line in 횡단보도.iloc:
        _,obj = line
        if geo_obj.contains(obj):
            cnt+=1
    return cnt

data["횡단보도_sum"] = data["geometry"].map(cross_walk)

> __사고 격자 주변 차량 등록 수 (200m)__  
(col_name = 차량등록_count_200)

In [ ]:
def car_registered(geo_obj):
    
    global 차량등록현황_격자
    
    y,x = geo_obj.centroid.coords[0]
    cnt=0
    for line in 차량등록현황_격자[차량등록현황_격자.car_cnt>0].iloc:
        _,val,obj_ = line

        if haversine((x,y),(obj_.centroid.coords[0][::-1]),unit="m")<=200:
            cnt+=val
    return cnt

data["차량등록_count_200"] = data["geometry"].map(car_registered)

> __사고 격자 주변 주정차 단속 건수 (300m)__  
(col_name = 주정차_count_300)

In [ ]:
def joojeong(geo_obj):
    
    global 주정차단속
    
    cnt=0
    for line in 주정차단속.iloc:
        y,x = line[-3],line[-2]

        if haversine((x,y),(geo_obj.centroid.coords[0][::-1]),unit="m")<=300:
            cnt+=1
    return cnt

data["주정차_count_300"] = data["geometry"].map(joojeong)

> __사고 격자 주변 과속방지턱 수 (100m , 200m , 300m)__  
(col_name = 방지턱_count_100 , 방지턱_count_200 , 방지턱_count_300)

In [ ]:
def count_in_grid(geo_obj,table,lon_col,lat_col,rad):
    cnt=0
    y,x = geo_obj.centroid.coords[0]
    
    for line in table.iloc:
        tar_x,tar_y = line[lat_col],line[lon_col]
        if haversine((x,y),(tar_x,tar_y),unit="m")<=rad:
            cnt+=1
        
    return cnt

data["방지턱_count_300"] = data["geometry"].map(lambda x :count_in_grid(x,과속방지턱,"설치위치_경도","설치위치_위도",300))
data["방지턱_count_200"] = data["geometry"].map(lambda x :count_in_grid(x,과속방지턱,"설치위치_경도","설치위치_위도",200))
data["방지턱_count_100"] = data["geometry"].map(lambda x :count_in_grid(x,과속방지턱,"설치위치_경도","설치위치_위도",100))

> __사고 격자 주변 무인교통단속카메라 수 (300m)__  
(col_name = 무인교통단속카메라_count_300)

In [ ]:
data["무인교통단속카메라_count_300"] = data["geometry"].map(lambda x :count_in_grid(x,무인교통단속카메라,"설치위치_경도","설치위치_위도",300))

> __사고 격자 주변 버스정류장 수 (100m , 200m , 300m)__  
(col_name = 버스정류장_count_100 , 버스정류장_count_200 , 버스정류장_count_300)

In [ ]:
data["버스정류장_count_300"] = data["geometry"].map(lambda x :count_in_grid(x,버스정류장,"정류장 위치_경도","정류장 위치_위도",300))
data["버스정류장_count_200"] = data["geometry"].map(lambda x :count_in_grid(x,버스정류장,"정류장 위치_경도","정류장 위치_위도",200))
data["버스정류장_count_100"] = data["geometry"].map(lambda x :count_in_grid(x,버스정류장,"정류장 위치_경도","정류장 위치_위도",100))

> __사고 격자 주변 초등학교 수 (100m , 200m , 300m)__  
(col_name = 초등학교_count_100 , 초등학교_count_200 , 초등학교_count_300)

In [ ]:
data["초등학교_count_300"] = data["geometry"].map(lambda x :count_in_grid(x,초등학교위치정보,"학교위치_경도","학교위치_위도",300))
data["초등학교_count_200"] = data["geometry"].map(lambda x :count_in_grid(x,초등학교위치정보,"학교위치_경도","학교위치_위도",200))
data["초등학교_count_100"] = data["geometry"].map(lambda x :count_in_grid(x,초등학교위치정보,"학교위치_경도","학교위치_위도",100))

> __사고 격자 주변 어린이집 수 (100m , 200m , 300m)__  
(col_name = 어린이집_count_100 , 어린이집_count_200 , 어린이집_count_300)

In [ ]:
어린이집_현황 = 어린이집_유치원현황[어린이집_유치원현황["시설구분"]=="어린이집"]
data["어린이집_count_300"] = data["geometry"].map(lambda x :count_in_grid(x,어린이집_현황,"시설위치_경도","시설위치_위도",300))
data["어린이집_count_200"] = data["geometry"].map(lambda x :count_in_grid(x,어린이집_현황,"시설위치_경도","시설위치_위도",200))
data["어린이집_count_100"] = data["geometry"].map(lambda x :count_in_grid(x,어린이집_현황,"시설위치_경도","시설위치_위도",100))

> __사고 격자 주변 유치원 수 (100m , 200m , 300m)__  
(col_name = 유치원_count_100 , 유치원_count_200 , 유치원_count_300)

In [ ]:
유치원_현황 = 어린이집_유치원현황[어린이집_유치원현황["시설구분"]=="유치원"]
data["유치원_count_300"] = data["geometry"].map(lambda x :count_in_grid(x,유치원_현황,"시설위치_경도","시설위치_위도",300))
data["유치원_count_200"] = data["geometry"].map(lambda x :count_in_grid(x,유치원_현황,"시설위치_경도","시설위치_위도",200))
data["유치원_count_100"] = data["geometry"].map(lambda x :count_in_grid(x,유치원_현황,"시설위치_경도","시설위치_위도",100))

> __학습데이터 저장__

In [ ]:
data.describe()

In [ ]:
data.to_file(os.path.join(path,'data_for_task1.geojson'),driver='GeoJSON')
data_df = pd.DataFrame(data)
data_df.to_csv(os.path.join(path,'data_for_task1.csv'),encoding="utf-8",index=False)

# [과제 2] 모델링을 위한 feature column 생성 : 총 17개 칼럼

__어린이보호구역 데이터를 기본으로 칼럼을 누적__

In [5]:
data2 = 어린이보호구역.copy()
data2.head(3)

,시설종류,시설명,CCTV설치여부,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도
0,특수학교,성심학교,N,0,25,127.017521,37.176762
1,초등학교,가수초등학교,Y,1,12,127.047957,37.148103
2,초등학교,광성초등학교,Y,2,12,127.039709,37.184197


> __보호구역 주변 어린이 교통사고 건수 (300m)__  
(col_name = 어린이교통사고횟수)

In [6]:
def check_accident(row):
    global 어린이교통사고_격자
    lat = row['보호구역_위도']
    lng = row['보호구역_경도']
    cnt = 0
    for grid in 어린이교통사고_격자[어린이교통사고_격자["accident_cnt"]>0].iloc:
        if haversine((lat, lng), grid['geometry'].centroid.coords[0][::-1], unit="m") <= 300:
            cnt+=grid['accident_cnt']
    return cnt

data2['어린이교통사고횟수']=data2.apply(check_accident, axis=1)

> __보호구역 주변 주택 수 (300m)__  
(col_name = 주택수)

In [7]:
def check_house(row):
    global 주택
    lat = row['보호구역_위도']
    lng = row['보호구역_경도']
    cnt = 0
    for grid in 주택.iloc:
        if haversine((lat, lng), grid['geometry'].centroid.coords[0][::-1], unit="m") <= 300:
            cnt+=1
    return cnt

주택 = 도로명주소_건물[(도로명주소_건물['BDTYP_CD'] =='01001')|(도로명주소_건물['BDTYP_CD'] =='02001')|(도로명주소_건물['BDTYP_CD'] =='01003')]

data2['주택수']=data2.apply(check_house, axis=1)

> __보호구역 주변 과속방지턱 수 (300m)__  
(col_name = 과속방지턱수)

In [8]:
def check_bump(row):
    global 과속방지턱
    lat = row['보호구역_위도']
    lng = row['보호구역_경도']
    cnt=0
    
    for line in 과속방지턱.iloc:
        tar_x,tar_y = line['설치위치_위도'],line['설치위치_경도']       
        if haversine((lat, lng),(tar_x, tar_y), unit="m") <= 300:
            cnt+=1           
    return cnt

data2['과속방지턱수'] = data2.apply(check_bump,axis=1)

> __보호구역 내부 최대속도 별 도로수 (최대속도 : 30,50,60,70,80)__  
(col_name = 최대속도30 , 최대속도50 , 최대속도60 , 최대속도70 , 최대속도80)

In [9]:
def check_lanes(row,speed):
    global 상세도로망
    lat = row['보호구역_위도']
    lng = row['보호구역_경도']
    cnt = 0
    for grid in 상세도로망[상세도로망.max_speed==str(speed)].iloc:
        if haversine((lat, lng), grid['geometry'].centroid.coords[0][::-1], unit="m") <= 300:
            cnt+=1
    return cnt

data2['최대속도30']=data2.apply(lambda x:check_lanes(x,30), axis=1)
data2['최대속도50']=data2.apply(lambda x:check_lanes(x,50), axis=1)
data2['최대속도60']=data2.apply(lambda x:check_lanes(x,60), axis=1)
data2['최대속도70']=data2.apply(lambda x:check_lanes(x,70), axis=1)
data2['최대속도80']=data2.apply(lambda x:check_lanes(x,80), axis=1)

> __보호구역 주변 무인교통단속 카메라 수 (100m)__  
(col_name = 주정차단속CCTV대수)

In [10]:
def check_parkingcamera(row):
    global 무인교통단속카메라
    lat = row['보호구역_위도']
    lng = row['보호구역_경도']
    cnt = 0
    
    for row2 in 무인교통단속카메라.iloc:
        if haversine((lat, lng), (row2['설치위치_위도'], row2['설치위치_경도']), unit="m") <= 100:
            cnt+=1
    return cnt

data2['주정차단속CCTV대수']=data2.apply(check_parkingcamera, axis=1)

> __보호구역 내부 평균 유동인구 수 (300m)__  
(col_name = 평균유동인구)

In [11]:
def check_movingppl(row):
    global 평균유동인구
    lat = row['보호구역_위도']
    lng = row['보호구역_경도']
    cnt = 0
    
    for row2 in 평균유동인구.iloc:
        if haversine((lat, lng), (row2['lat'], row2['lon']), unit="m") <= 300:
            cnt += row2['평균유동인구']
    return cnt

data2['평균유동인구']=data2.apply(check_movingppl, axis=1)

> __보호구역 주변 주정차 단속 건수 (300m)__  
(col_name = 이동형CCTV단속횟수)

In [12]:
def check_movingcctv(row):
    global 주정차단속
    lat = row['보호구역_위도']
    lng = row['보호구역_경도']
    cnt = 0
    for row2 in 주정차단속[주정차단속["스쿨존여부"]=="Y"].iloc:
        if haversine((lat, lng), (row2['단속위치_위도'], row2['단속위치_경도']), unit="m") <= 300:
            cnt += 1
    return cnt

data2['이동형CCTV단속횟수']=data2.apply(check_movingcctv, axis=1)

> __보호구역 주변 거주중인 유소년 인구수 (300m)__  
(col_name = 거주인구_유소년)

In [13]:
def check_kids(row):
    global 거주인구격자_유소년
    lat = row['보호구역_위도']
    lng = row['보호구역_경도']
    cnt = 0
    for grid in 거주인구격자_유소년[거주인구격자_유소년["val"]>0].iloc:
        if haversine((lat, lng), grid['geometry'].centroid.coords[0][::-1], unit="m") <= 300:
            cnt+=grid['val']
    return cnt

data2['거주인구_유소년']=data2.apply(check_kids, axis=1)

> __보호구역 내부 등록된 차량 수 (300m)__  
(col_name = 등록차량대수)

In [14]:
def check_cars(row):
    global 차량등록현황_격자
    lat = row['보호구역_위도']
    lng = row['보호구역_경도']
    cnt = 0
    for grid in 차량등록현황_격자[차량등록현황_격자["car_cnt"]>0].iloc:
        if haversine((lat, lng), grid['geometry'].centroid.coords[0][::-1], unit="m") <= 300:
            cnt+=grid['car_cnt']
    return cnt

data2['등록차량대수']=data2.apply(check_cars, axis=1)

> __학습데이터 저장__

In [15]:
data2.describe()

,CCTV설치대수,보호구역도로폭,보호구역_경도,보호구역_위도,어린이교통사고횟수,주택수,과속방지턱수,최대속도30,최대속도50,최대속도60,최대속도70,최대속도80,주정차단속CCTV대수,평균유동인구,이동형CCTV단속횟수,거주인구_유소년,등록차량대수
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.00000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,1.155556,8.422222,127.062605,37.156712,8.400000,133.833333,0.500000,1.755556,2.377778,4.433333,0.30000,2.644444,0.266667,622.285441,1.955556,519.466667,1641.033333
std,0.806032,4.773470,0.015249,0.018001,7.026115,112.145997,0.951049,3.029114,5.470201,3.741808,1.26713,3.895578,0.492642,421.468969,8.309885,458.691133,1183.662251
min,0.000000,3.000000,127.017521,37.127317,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,27.610148,0.000000,0.000000,23.000000
25%,1.000000,6.000000,127.053071,37.141175,3.000000,37.750000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,297.546506,0.000000,159.000000,793.250000
50%,1.000000,6.000000,127.065259,37.154981,6.000000,102.500000,0.000000,0.000000,0.000000,4.000000,0.00000,0.000000,0.000000,568.996966,0.000000,378.000000,1466.000000
75%,1.750000,12.000000,127.072374,37.164735,14.000000,192.000000,1.000000,3.000000,2.000000,6.000000,0.00000,5.000000,0.000000,849.347125,1.000000,781.500000,2072.250000
max,3.000000,25.000000,127.090958,37.197877,27.000000,495.000000,3.000000,17.000000,28.000000,15.000000,7.00000,18.000000,2.000000,1800.153568,57.000000,1671.000000,5232.000000


In [16]:
data2.to_csv(os.path.join(path,'data_for_task2.csv'),encoding="utf-8",index=False)